In [1]:
#import openai
import google.generativeai as genai
import langchain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
import pinecone

e:\vdblangchain\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()



True

In [3]:
import os

In [4]:
#reading the document

def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    print(documents)
    return documents
   

In [5]:
doc=read_doc('documents/')

[Document(metadata={'source': 'documents\\Microsoft 2023 Annual Report.pdf', 'page': 0}, page_content="Home\nShareholder Letter\nFinancial R eview\nBusiness\nGeneral\nHuman Capital R esources\nOperating Segments\nOperations\nResearch & Development\nDistribution, Sales & Marketing\nLicensing Options\nCustomers\nAvailable Information\nDiscussion & Analysis\nMarket Risk\nIncome S tatements\nBalance Sheets\nCash Flows S tatements\nStockholders' Equity\nNotes\nAccounting P olicies\nEarnings P er Share\nOther Income (Expense), Net\nInvestments\nDerivatives\nInventories\nProperty and Equipment\nBusiness Combinations\nGoodwill\nIntangible Assets\nDebt\nIncome T axes\nUnearned R evenue\nLeases\nContingencies\nStockholders' Equity\nAccumulated Other Comprehensive Income (Loss)\nEmployee S tock and Savings Plans\nSegment Information & Geo Data\nAuditor's R eport\nControls & Procedures\nDirectors & Officers\nInvestor R elations\nAnnual R eport 2023\nContinue to Shareholder LetterAnnual R epor t 20

In [6]:
len(doc)
print(doc[10])

page_content='We are committed to designing and marketing first-party devices to help drive innovation, create new device categories, and
stimulate demand in the Windows ecosystem. The Surface family includes Surface Pro, Surface Laptop, and other Surface products.
Microsoft continues to invest in gaming content, community, and cloud services. W e have broadened our approach to how we think
about gaming end-to-end, from the way games are created and distributed to how they are played, including subscription services
like Xbox Game P ass and new devices from third-party manufacturers so players can engage across PC, console, and mobile. In
January 2022, we announced plans to acquire Activision Blizzard, Inc., a leader in game development and an interactive entertainment
content publisher.
Our Futur e Oppor tunity
We are focused on helping customers use the breadth and depth of the Microsoft Cloud to get the most value out of their digital
spend while leading the new AI wave across our s

In [7]:
#convert big docs into chunks!! chonk

def chunk_data(docs,chunk_size=800, chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [8]:
chunks=chunk_data(doc)
print(chunks)

[Document(metadata={'source': 'documents\\Microsoft 2023 Annual Report.pdf', 'page': 0}, page_content="Home\nShareholder Letter\nFinancial R eview\nBusiness\nGeneral\nHuman Capital R esources\nOperating Segments\nOperations\nResearch & Development\nDistribution, Sales & Marketing\nLicensing Options\nCustomers\nAvailable Information\nDiscussion & Analysis\nMarket Risk\nIncome S tatements\nBalance Sheets\nCash Flows S tatements\nStockholders' Equity\nNotes\nAccounting P olicies\nEarnings P er Share\nOther Income (Expense), Net\nInvestments\nDerivatives\nInventories\nProperty and Equipment\nBusiness Combinations\nGoodwill\nIntangible Assets\nDebt\nIncome T axes\nUnearned R evenue\nLeases\nContingencies\nStockholders' Equity\nAccumulated Other Comprehensive Income (Loss)\nEmployee S tock and Savings Plans\nSegment Information & Geo Data\nAuditor's R eport\nControls & Procedures\nDirectors & Officers\nInvestor R elations\nAnnual R eport 2023\nContinue to Shareholder LetterAnnual R epor t 20

In [9]:
# Print type and attributes of a chunk
for chunk in chunks:
    print(type(chunk))
    print(dir(chunk))
    break  # Print only the first chunk for inspection


<class 'langchain_core.documents.base.Document'>
['Config', '__abstractmethods__', '__annotations__', '__class__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_repr__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validators__', '_abc_impl', '_calculate_keys', '_copy_and_set_values', '_decompose_class', '_enforce_dict

In [10]:
import google.generativeai as genai
from langchain.embeddings import GooglePalmEmbeddings

# Set up the API key
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

# Create the embeddings object
embeddings = GooglePalmEmbeddings()

# embedding for each chunk!
embeddings_list = [embeddings.embed_query(chunk.page_content) for chunk in chunks]

# Print the chunks and their embeddings
for chunk, vec in zip(chunks, embeddings_list):
    print("Chunk:", chunk.page_content)
    print("Embedding:", vec)
    print()

Chunk: Home
Shareholder Letter
Financial R eview
Business
General
Human Capital R esources
Operating Segments
Operations
Research & Development
Distribution, Sales & Marketing
Licensing Options
Customers
Available Information
Discussion & Analysis
Market Risk
Income S tatements
Balance Sheets
Cash Flows S tatements
Stockholders' Equity
Notes
Accounting P olicies
Earnings P er Share
Other Income (Expense), Net
Investments
Derivatives
Inventories
Property and Equipment
Business Combinations
Goodwill
Intangible Assets
Debt
Income T axes
Unearned R evenue
Leases
Contingencies
Stockholders' Equity
Accumulated Other Comprehensive Income (Loss)
Employee S tock and Savings Plans
Segment Information & Geo Data
Auditor's R eport
Controls & Procedures
Directors & Officers
Investor R elations
Annual R eport 2023
Continue to Shareholder LetterAnnual R epor t 2023
home   8/3/24, 12:09 PM Microsoft 2023 Annual Report
https://www.microsoft.com/investor/reports/ar23/ 1/68
Embedding: [0.011073941, -0.00

In [11]:
print(embeddings)

client=<module 'google.generativeai' from 'e:\\vdblangchain\\myenv\\Lib\\site-packages\\google\\generativeai\\__init__.py'> google_api_key=None model_name='models/embedding-gecko-001' show_progress_bar=False


In [12]:
vectors=embeddings.embed_query("what is it")
len(vectors)
#for dimensions we use len
#not word2vec, openai more advanced, vector size

768

In [14]:
from pinecone import Pinecone, ServerlessSpec, PodSpec
import uuid

pc = Pinecone(
    api_key="af916129-5961-4c14-b3ab-a82d6ce7dcad",
    environment="us-east-1",
    serverless_spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

index_name = "newlangchain"

# Check if the index exists, if not, create it
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,  # adjust this to match your embedding dimension
        metric='cosine',
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

index = pc.Index(index_name)

# Function to ensure ID is a valid string
def ensure_valid_id(id):
    if id is None:
        return str(uuid.uuid4())  # Generate a random UUID if id is None
    return str(id)  # Convert to string if it's not already

# Assuming 'documents' is your list of documents and 'embeddings' is your embedding model
vectors = [(ensure_valid_id(d.id), embeddings.embed_query(d.page_content), d.metadata) for d in chunks]

# Upsert in batches
batch_size = 100  # Adjust this based on your needs
for i in range(0, len(vectors), batch_size):
    batch = vectors[i:i+batch_size]
    index.upsert(vectors=batch)

In [15]:
from langchain.schema import Document

def retrieve_query(query, k=2):
    query_embedding = embeddings.embed_query(query)
    results = index.query(
        vector=query_embedding,
        top_k=k,
        include_metadata=True
    )
    
    # Convert results to Document objects
    documents = [
        Document(page_content=result.metadata.get('text', ''), metadata=result.metadata)
        for result in results.matches
    ]
    
    return documents

In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_texts([chunk.page_content for chunk in chunks], embeddings)

# Define retriever
retriever = vectorstore.as_retriever()
# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest", temperature=0.5)

# Create the conversational chain
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,  # You need to define your retriever
    chain_type="stuff"
)

In [18]:
def retrieve_answers(query, chat_history=None):
    doc_search = retrieve_query(query)
    
    if chat_history is None:
        chat_history = []
    
    # Prepare inputs based on the chain's expected input keys
    inputs = {
        "question": query,
        "chat_history": chat_history
    }
    
    # Only add input_documents if it's in the chain's input keys
    if "input_documents" in chain.input_keys:
        inputs["input_documents"] = doc_search
    
    # Run the chain with the prepared inputs
    response = chain(inputs)
    
    # Extract the answer from the response
    # Adjust this based on your chain's output keys
    if len(chain.output_keys) == 1:
        return response[chain.output_keys[0]]
    else:
        return response



In [22]:
ourquery = "what are the major changes implemented in 2023. was this a financially successful year?"
answer = retrieve_answers(ourquery)  # This will use an empty chat history
print(answer)

# If you want to use a chat history in subsequent calls:
chat_history = [("Previous question", "Previous answer")]
answer = retrieve_answers(ourquery, chat_history=chat_history)
print(answer)

The text provides a detailed breakdown of Microsoft's financial performance in 2023, including revenue growth in various segments like Microsoft Cloud, Office products, LinkedIn, and Dynamics. It also mentions a significant increase in operating expenses, driven by factors like employee severance and investments in cloud engineering. 

However, the document lacks information about specific "major changes" implemented in 2023 beyond financial results.  

While the report highlights positive growth in several areas, it doesn't offer a conclusive statement about whether the year was financially successful overall.  

To determine overall financial success, you'd need to consider factors like net income, profitability ratios, and comparisons to previous years or industry benchmarks, which aren't explicitly stated in this excerpt. 

This question cannot be answered from the given text. This excerpt of the Microsoft 2023 Annual Report focuses on financial data and does not describe specific 